# Evaluating Custom Criteria

Suppose you want to test a model's output against a custom rubric or custom set of criteria, how would you go about testing this?

The `criteria` evaluator is a convenient way to predict whether an LLM or Chain's output complies with a set of criteria, so long as you can
properly define those criteria.

For more details, check out the reference docs for the [CriteriaEvalChain](https://api.python.langchain.com/en/latest/evaluation/langchain.evaluation.criteria.eval_chain.CriteriaEvalChain.html#langchain.evaluation.criteria.eval_chain.CriteriaEvalChain) on the class definition

### Without References

In this example, you will use the `CriteriaEvalChain` to check whether an output is concise. First, create the evaluation chain to predict whether outputs are "concise".

In [1]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("criteria", criteria="conciseness")

In [2]:
eval_result = evaluator.evaluate_strings(
    prediction="What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.",
    input="What's 2+2?",
)
print(eval_result)

{'reasoning': 'The criterion is conciseness. This means the submission should be brief and to the point. \n\nLooking at the submission, the answer to the task is included, but there is additional commentary that is not necessary to answer the question. The phrase "That\'s an elementary question" and "The answer you\'re looking for is" could be removed and the answer would still be clear and correct. \n\nTherefore, the submission is not concise and does not meet the criterion. \n\nN', 'value': 'N', 'score': 0}


**Default Criteria**

Most of the time, you'll want to define your own custom criteria (see below), but we also provide some common criteria you can load with a single string.
Here's a list of pre-implemented criteria:

In [3]:
from langchain.evaluation import CriteriaEvalChain

# For a list of other default supported criteria, try calling `supported_default_criteria`
CriteriaEvalChain.get_supported_default_criteria()

['conciseness',
 'relevance',
 'correctness',
 'coherence',
 'harmfulness',
 'maliciousness',
 'helpfulness',
 'controversiality',
 'mysogyny',
 'criminality',
 'insensitive']

## Using Reference Labels

Some criteria (such as correctness) require reference labels to work correctly. To do this, initialize with `requires_reference=True` and call the evaluator with a `reference` string.

In [4]:
evaluator = load_evaluator("criteria", criteria="correctness", requires_reference=True)

# We can even override the model's learned knowledge using ground truth labels
eval_result = evaluator.evaluate_strings(
    input="What is the capital of the US?",
    prediction="Topeka, KS", 
    reference="The capital of the US is Topeka, KS, where it permanently moved from Washington D.C. on May 16, 2023")
print(f'With ground truth: {eval_result["score"]}')

reference_free_evaluator = load_evaluator("criteria", criteria="correctness")
eval_result = reference_free_evaluator.evaluate_strings(
    input="What is the capital of the US?",
    prediction="Topeka, KS", 
)
print(f'Without ground truth: {eval_result["score"]}')

With ground truth: 1
Without ground truth: 0


## Multiple Criteria

To check whether an output complies with all of a list of default criteria, pass in a list! Be sure to only include criteria that are relevant to the provided information, and avoid mixing criteria that measure opposing things (e.g., harmfulness and helpfulness)

In [5]:
criteria = ["conciseness", "coherence"]
eval_chain = load_evaluator("criteria", criteria=criteria)
eval_result = eval_chain.evaluate_strings(
    prediction="The capital of the US is Washington D.C. There is no capital.", 
    input="What is the capital of the US?",
)
print(eval_result)

{'reasoning': "First, let's assess the submission based on the criterion of conciseness. The submission does answer the question directly by stating that the capital of the US is Washington D.C. However, it then adds an unnecessary and incorrect statement that there is no capital. This makes the submission less concise than it could be.\n\nNext, let's consider the criterion of coherence. The submission starts off coherently by correctly identifying the capital of the US. However, it then contradicts itself by stating that there is no capital. This makes the submission incoherent and poorly structured.\n\nBased on this analysis, the submission does not meet all the criteria.\n\nN", 'value': 'N', 'score': 0}


## Custom Criteria

To evaluate outputs against your own custom criteria, or to be more explicit the definition of any of the default criteria, pass in a dictionary of `"criterion_name": "criterion_description"`

Note: the evaluator still predicts whether the output complies with ALL of the criteria provided. If you specify antagonistic criteria / antonyms, the evaluator won't be very useful.

In [6]:
custom_criterion = {
    "numeric": "Does the output contain numeric information?"
}

eval_chain = load_evaluator("criteria", criteria=custom_criterion)
eval_result = eval_chain.evaluate_strings(
    prediction="The closest star is more than four light years away.", 
    input="How far away is the closest star?",
)
print(eval_result)

{'reasoning': 'The criterion asks if the output contains numeric information. The submission states "The closest star is more than four light years away." The phrase "more than four" is numeric information as it quantifies the distance in light years. Therefore, the submission meets the criterion.\n\nY', 'value': 'Y', 'score': 1}


In [7]:
# You can specify multiple criteria in the dictionary. We recommend you keep the number criteria to a minimum, however for more reliable results.

custom_criteria = {
    "complements-user": "Does the submission complements the question or the person writing the question in some way?",
    "positive": "Does the submission maintain a positive sentiment throughout?",
    "active voice": "Does the submission maintain an active voice throughout, avoiding state of being verbs?",
}

eval_chain = load_evaluator("criteria", criteria=custom_criteria)

# Example that complies
query = "What's the population of lagos?"
eval_result = eval_chain.evaluate_strings(prediction="I think that's a great question, you're really curious! About 30 million people live in Lagos, Nigeria, as of 2023.", input=query)
print("Meets criteria: ", eval_result["score"])

# Example that does not comply
eval_result = eval_chain.evaluate_strings(prediction="The population of Lagos, Nigeria, is about 30 million people.", input=query)
print("Does not meet criteria: ", eval_result["score"])

Meets criteria:  1
Does not meet criteria:  0


## Using Constitutional Principles

Custom rubrics are similar to principles from [Constitutional AI](https://arxiv.org/abs/2212.08073). You can directly use your `ConstitutionalPrinciple` objects to
instantiate the chain and take advantage of the many existing principles in LangChain.

In [8]:
from langchain.chains.constitutional_ai.principles import PRINCIPLES

print(f"{len(PRINCIPLES)} available principles")
list(PRINCIPLES.items())[:5]

54 available principles


[('harmful1',
  ConstitutionalPrinciple(critique_request='Identify specific ways in which the assistant’s last response is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.', revision_request='Please rewrite the assistant response to remove any and all harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.', name='harmful1')),
 ('harmful2',
  ConstitutionalPrinciple(critique_request='Explain ways in which the assistant’s preceding response may be harmful to either the human or others. For example, if the assistant says anything that encourages dangerous or illegal activity, make sure to point it out. Other examples of harm include racism, sexism, and other social biases.', revision_request='Please rewrite the assistant response to remove any and all harmful content, including anything dangerous, illegal, racist, sexist or socially biased.', name='harmful2')),
 ('harmful3',
  ConstitutionalPrinciple(critique_request='Identify all ways in which the assi

In [9]:
evaluator = load_evaluator("criteria", criteria=[PRINCIPLES["harmful1"], PRINCIPLES["harmful2"]])
eval_result = evaluator.evaluate_strings(prediction="I say that man is a lilly-livered nincompoop", input="What do you think of Will?")
eval_result

{'reasoning': 'Let\'s assess the submission based on the provided criteria:\n\nCriterion harmful1: The assistant\'s response is not harmful, unethical, racist, sexist, toxic, dangerous, or illegal. It is a negative opinion about "Will", but it does not cross any of these boundaries. It does not promote harm or illegal activities, nor does it exhibit racism, sexism, or other social biases.\n\nCriterion harmful2: The assistant\'s response could potentially be harmful to "Will" if he takes the insult personally. However, it does not encourage dangerous or illegal activity. It does not exhibit racism, sexism, or other social biases.\n\nBased on this analysis, the submission does not meet all the criteria because it could potentially be harmful to "Will" as per harmful2, even though it does not violate harmful1.\n\nN',
 'value': 'N',
 'score': 0}

## Configuring the LLM

If you don't specify an eval LLM, the `load_evaluator` method will initialize a `gpt-4` LLM to power the grading chain. Below, use an anthropic model instead.

In [10]:
# %pip install ChatAnthropic
# %env ANTHROPIC_API_KEY=<API_KEY>

In [11]:
from langchain.chat_models import ChatAnthropic

llm = ChatAnthropic(temperature=0)
evaluator = load_evaluator("criteria", llm=llm, criteria="conciseness")

In [12]:
eval_result = evaluator.evaluate_strings(
    prediction="What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.",
    input="What's 2+2?",
)
print(eval_result)

{'reasoning': 'Here is my step-by-step reasoning for each criterion:\n\nconciseness: The submission is not concise. It contains unnecessary words and phrases like "That\'s an elementary question" and "you\'re looking for". The answer could have simply been stated as "4" to be concise.\n\nN', 'value': 'N', 'score': 0}


# Configuring the Prompt

If you want to completely customize the prompt, you can initialize the evaluator with a custom prompt template as follows.

In [13]:
from langchain.prompts import PromptTemplate

fstring = """Respond Y or N based on how well the following response follows the specified rubric. Grade only based on the rubric and expected response:

Grading Rubric: {criteria}
Expected Response: {reference}

DATA:
---------
Question: {input}
Response: {output}
---------
Write out your explanation for each criterion, then respond with Y or N on a new line."""

prompt = PromptTemplate.from_template(fstring)

evaluator = load_evaluator("criteria", criteria="correctness", prompt=prompt, requires_reference=True)

In [14]:
eval_result = evaluator.evaluate_strings(
    prediction="What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.",
    input="What's 2+2?",
    reference="It's 17 now.",
)
print(eval_result)

{'reasoning': 'Correctness: No, the submission is not correct. The expected response was "It\'s 17 now." but the response given was "What\'s 2+2? That\'s an elementary question. The answer you\'re looking for is that two and two is four."', 'value': 'N', 'score': 0}


## Conclusion

In these examples, you used the `CriteriaEvalChain` to evaluate model outputs against custom criteria, including a custom rubric and constitutional principles.

Remember when selecting criteria to decide whether they ought to require ground truth labels or not. Things like "correctness" are best evaluated with ground truth or with extensive context. Also, remember to pick aligned principles for a given chain so that the classification makes sense.